# MAVIS Astrometric Simulator

## Phase A Version
March 2020

## Todo
1. Update the PSDdatabase

# Imports

In [ ]:
# Standard
import numpy as np
import matplotlib.pyplot as plt
import sys
import os

# Astropy
from astropy.io import fits, ascii
from astropy.table import Table

# Project-specific/
sys.path.append('../src')
import mavissimim.rampup
import mavissimim.addconstantsky
import mavissimim.addnoise
import mavissimim.trimimage
import mavissimim.input_parameters as input_par
from rebin import rebin

from mavissimim.Source import Source
from mavissimim.AOGaussGrid import AOGaussGrid
from mavissimim.SeeingGrid import SeeingGrid
from mavissimim.InputCoo import InputCoo


# Keep track of the simulations

## Follow the conventions in the following table
| Simulation       | TT Jitter | PSF Variability | Static Dist |
|------------------|-----------|-----------------|-------------|
| 1 (Realistic I, no IMBH)  | On-Average (6mas)   | On              | On          |
| 2 (Realistic II, no IMBH) | On-Worst (~30mas, amplified 25_25_25) | On              | On          |
| 3                | On-Average(~6mas, 2 * amplified 25_25_25)   | Off             | Off         |
| 4                | On-Worst(30_10_10)  | Off             | Off         |
| 5                | Off       | On              | Off         |
| 6                | Off       | Off             | On          |
| Worst            | On-Worst (10x amplified 25_25_25)         | On      | On (amplified x100, 8mas)
| 7 (Realistic III, IMBH)      | On-Average (~6mas, 2 * amplified 25_25_25)         | On              | On          |
| 8 (Realisitic IV, IMBH)      | On-Worst (~30mas, 5 * amplified 25_25_25)  | On | On

## Updated - Aug 17/20
All have 1500 M IMBH at centre, time 0

| Simulation       | TT Jitter | Constellation | PSF Variability |
|------------------|-----------|---------------|-----------------|
| I  | Off   | Off              | Off          |
| II| On    | 10_10_10         | On           |
| III | On    | 30_40_10         | On           |
| IV  | On    | 40_40_40         | On           |

In [ ]:
exp_time = 3600

fits_save_path = ('/Users/smonty/Dropbox/ANUResearch/AO/AstrometryPaper/IMBH/SimI/') #% str(input_par.nbody_yr))
coo_save_path = ('/Users/smonty/Dropbox/ANUResearch/AO/AstrometryPaper/IMBH/SimI/') #% str(input_par.nbody_yr))

file_name = "StaticPSF_NoTTInput_75mas_11masJit%s" % str(exp_time)
print (input_par.fv_psf_path)
print (input_par.nbody_yr)
print (input_par.gauss_offset)


# Load the input data

In [ ]:
# Nbody input plus chosen exposure time
ngc3201 = input_par.nbody_in

In [ ]:
print (input_par.fv_psf_path)
print (input_par.gauss_sigma)
print (input_par.nbody_yr)
print (input_par.psf_core_rad_pix, input_par.seeing_core_rad_pix)

# Create the source object (to populate the image)

In [ ]:
source = Source(ngc3201, exp_time, static_dist=False, stat_amp=1, tt_var=False, tt_amp_fac=3.0, tt_static=False, tt_kern=0).main()
#self, mavis_src, exp_time, static_dist, stat_amp, tt_var, tt_amp_fac, tt_static, tt_kern

# Create the input catalogue to compare with DAOPhot

In [ ]:
input_coo = InputCoo(source).main()

# Create the convolved AO core grid and the Gaussian grid

In [ ]:
ao_field, gauss_field = AOGaussGrid(source, fv_psf=False).main()

# Use the large PSF to create a grid of seeing wings

In [ ]:
seeing_field = SeeingGrid(gauss_field).main()

# Make the final image

In [ ]:
image = ao_field + seeing_field

# Add sky background

In [ ]:
#image_plussky, sky_val = mavissimim.addconstantsky.add_constant_sky(image, exp_time)

# Add all noise

**Idea** = randomly generate a read_noise frame to simulate a bias image

In [ ]:
#image_plussky_rn = mavissimim.addnoise.add_read_noise(image_plussky, sky_val)
# Add shot noise, read noise and convert from electrons to ADU
image_allnoise = mavissimim.addnoise.add_all_noise(image, source.meta["exp_time"])

# Trim the final image and the input catalogue

In [ ]:
(image_plusskyrn_trim, trimmed_input_coo) = mavissimim.trimimage.trim_image(image_allnoise, input_coo)
#(image_plusskyrn_trim, trimmed_input_coo) = mavissimim.trimimage.trim_image(image_plussky_rn, input_coo)


# Save the final image and input catalogue

In [ ]:
final_image = np.array(image_plusskyrn_trim, dtype="float32")
hdu = fits.PrimaryHDU(final_image)
hdul = fits.HDUList([hdu])
hdul.writeto(fits_save_path + file_name + ".fits", overwrite=True)

In [ ]:
ao_image = np.array(ao_frame_trim, dtype="float32")
hdu = fits.PrimaryHDU(ao_image)
hdul = fits.HDUList([hdu])
hdul.writeto(fits_save_path + file_name + 'AOField.fits', overwrite=True)

In [ ]:
gauss_image = np.array(gauss_frame_trim, dtype="float32")
hdu = fits.PrimaryHDU(gauss_image)
hdul = fits.HDUList([hdu])
hdul.writeto(fits_save_path + file_name + 'GaussField.fits', overwrite=True)

In [ ]:
seeing_image = np.array(sky_plusskyrn_trim, dtype="float32")
hdu = fits.PrimaryHDU(seeing_image)
hdul = fits.HDUList([hdu])
hdul.writeto(fits_save_path + file_name + 'SeeField.fits', overwrite=True)

In [ ]:
ascii.write(trimmed_input_coo, coo_save_path + file_name + '.all', overwrite=True)

In [ ]:
print (trimmed_input_coo["CCD_Mapped_X"][0])
print (trimmed_input_coo["CCD_Mapped_Y"][0])
print (trimmed_input_coo["Flux"][0])